# Day 4 - Lab 2: Generating a CI/CD Pipeline

**Objective:** Use an LLM to generate all necessary configuration files to create an automated Continuous Integration (CI) pipeline for the FastAPI application using Docker and GitHub Actions.

**Estimated Time:** 75 minutes

**Introduction:**
A robust CI pipeline is the backbone of modern software development. It automatically builds and tests your code every time a change is made, catching bugs early and ensuring quality. In this lab, you will generate all the configuration-as-code artifacts needed to build a professional CI pipeline for our application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load our application code to provide context for the LLM. The AI needs to see our code's imports to generate an accurate `requirements.txt` file.

**Model Selection:**
Models that are good at understanding code and structured data formats like YAML are ideal. `gpt-4.1`, `o3`, or `codex-mini` are strong choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated configuration files.
- `clean_llm_output()`: To clean up the generated text and code.

In [2]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the application code from Day 3 to provide context
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-10-31 11:35:05,022 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating a `requirements.txt`

**Task:** Before we can build a Docker image, we need a list of our Python dependencies. Prompt the LLM to analyze your application code and generate a `requirements.txt` file.

**Instructions:**
1.  Write a prompt that provides the LLM with the source code of your FastAPI application (`app_code`).
2.  Instruct it to analyze the `import` statements and generate a list of all external dependencies (like `fastapi`, `uvicorn`, `sqlalchemy`). You should also ask it to include `pytest` for testing.
3.  The output should be formatted as a standard `requirements.txt` file.
4.  Save the artifact to the project's root directory.

In [3]:
# Detailed prompt to generate a requirements.txt file
requirements_prompt = f"""You are a Python dependency management expert. Analyze the following FastAPI application code and generate a complete, production-ready `requirements.txt` file.

**Application Code:**
```python
{app_code}
```

**Task Requirements:**

1. **Systematically analyze ALL import statements** in the code, including:
   - Direct imports (e.g., `import uvicorn`)
   - From imports (e.g., `from fastapi import FastAPI`)
   - Third-party packages (exclude standard library modules like `os`, `sys`, `datetime`, `enum`, `typing`, `logging`)

2. **Include ALL necessary dependencies:**
   - Core FastAPI dependencies: `fastapi`, `uvicorn[standard]` (the [standard] extra includes performance optimizations)
   - Pydantic and validation: `pydantic`, `pydantic-settings`, `email-validator` (required for EmailStr)
   - Database: `sqlalchemy`
   - Testing: `pytest`, `httpx` (required by FastAPI's TestClient)

3. **Version pinning strategy:**
   - Use compatibility version pinning (e.g., `fastapi>=0.104.0,<1.0.0`)
   - This allows bug fixes and minor updates while preventing breaking changes
   - For stable packages, use `~=` operator (e.g., `sqlalchemy~=2.0.0` means >=2.0.0, <2.1.0)

4. **Organization and formatting:**
   - Group dependencies by category with comments (Core, Database, Testing, etc.)
   - List packages alphabetically within each category
   - Include brief inline comments for non-obvious dependencies

5. **Best practices:**
   - DO NOT include standard library modules
   - Include transitive dependencies that are directly imported (like `email-validator`)
   - Ensure the file works for both development and CI/CD environments
   - Include pytest for the test suite

**Expected Output Format:**
Generate ONLY the contents of the requirements.txt file. No explanations, no markdown code blocks, no additional commentary. The output should be ready to save directly to a file.

Example of desired format:
```
# Core Framework
fastapi>=0.104.0,<1.0.0
uvicorn[standard]>=0.24.0,<1.0.0

# Database
sqlalchemy~=2.0.0

# Validation and Settings
pydantic>=2.0.0,<3.0.0
pydantic-settings>=2.0.0,<3.0.0
email-validator>=2.0.0,<3.0.0

# Testing
pytest>=7.4.0,<8.0.0
httpx>=0.25.0,<1.0.0
```

Generate the complete requirements.txt file now:"""

print("--- Generating requirements.txt ---")
if app_code:
    requirements_content = get_completion(requirements_prompt, client, model_name, api_provider)
    cleaned_reqs = clean_llm_output(requirements_content, language='text')
    print(cleaned_reqs)
    save_artifact(cleaned_reqs, "requirements.txt")
else:
    print("Skipping requirements generation because app code is missing.")

--- Generating requirements.txt ---
# Core Framework
fastapi>=0.111.0,<1.0.0
uvicorn[standard]>=0.29.0,<1.0.0

# Database
sqlalchemy~=2.0.29

# Validation and Settings
email-validator>=2.0.0,<3.0.0  # Required by Pydantic's EmailStr
pydantic>=2.7.0,<3.0.0
pydantic-settings>=2.2.0,<3.0.0

# Testing
httpx>=0.27.0,<1.0.0  # Required by FastAPI's TestClient
pytest>=8.0.0,<9.0.0


### Challenge 2 (Intermediate): Generating a `Dockerfile`

**Task:** Generate a multi-stage `Dockerfile` to create an optimized and secure container image for our application.

> **Tip:** Why a multi-stage Dockerfile? The first stage (the 'builder') installs all dependencies, including build-time tools. The final stage copies only the application code and the necessary installed packages. This results in a much smaller, more secure production image because it doesn't contain any unnecessary build tools.

**Instructions:**
1.  Write a prompt asking for a multi-stage `Dockerfile` for a Python FastAPI application.
2.  Specify the following requirements:
    * Use a slim Python base image (e.g., `python:3.11-slim`).
    * The first stage should install dependencies from `requirements.txt`.
    * The final stage should copy the installed dependencies and the application code.
    * The `CMD` should execute the application using `uvicorn`.
3.  Save the generated file as `Dockerfile` in the project's root.

In [4]:
# Detailed prompt to generate a multi-stage Dockerfile
dockerfile_prompt = """You are a Docker and containerization expert specializing in Python applications. Generate a production-ready, optimized multi-stage Dockerfile for a FastAPI application.

**Project Context:**
- FastAPI application with SQLAlchemy database (SQLite)
- Application code located in `artifacts/app/` directory
- Main application file: `artifacts/app/main.py`
- Dependencies listed in `requirements.txt` (already generated)
- Database file: `artifacts/app/onboarding.db`
- Application should run on port 8000

**Task Requirements:**

1. **Multi-Stage Build Structure:**
   - **Stage 1 (Builder)**: Install all Python dependencies and build wheels
     - Base image: `python:3.11-slim`
     - Install dependencies from `requirements.txt`
     - Use a virtual environment for dependency isolation
   - **Stage 2 (Runtime)**: Create minimal production image
     - Base image: `python:3.11-slim`
     - Copy only the installed dependencies (not build tools)
     - Copy application code
     - Set up runtime configuration

2. **Security Best Practices:**
   - Run as non-root user (create and use an `appuser`)
   - Use `--no-cache-dir` flag when installing pip packages to reduce image size
   - Set `PYTHONUNBUFFERED=1` environment variable for proper logging
   - Set `PYTHONDONTWRITEBYTECODE=1` to prevent .pyc file generation

3. **Optimization Requirements:**
   - Leverage Docker layer caching (copy requirements.txt before application code)
   - Minimize final image size by excluding build tools
   - Use `.dockerignore` patterns in comments (for user reference)
   - Set appropriate WORKDIR paths

4. **Application Configuration:**
   - WORKDIR should be `/app`
   - Copy the entire `artifacts/app/` directory to `/app/`
   - Ensure the database directory is writable
   - EXPOSE port 8000
   - CMD should run: `uvicorn main:app --host 0.0.0.0 --port 8000`
   - Note: Since app files are in artifacts/app/, the import path is `main:app`

5. **Directory Structure Awareness:**
   - Application structure:
     ```
     artifacts/
       app/
         main.py          # FastAPI app
         onboarding.db    # SQLite database
     requirements.txt     # Dependencies
     ```

**Expected Output Format:**
Generate ONLY the Dockerfile contents. No explanations before or after, no markdown code blocks wrapper. Include helpful inline comments in the Dockerfile itself explaining each section. The output should be ready to save directly as a Dockerfile.

**Example of desired format and style:**
```dockerfile
# Stage 1: Builder - Install dependencies
FROM python:3.11-slim AS builder

# Set working directory
WORKDIR /app

# Copy requirements first for layer caching
COPY requirements.txt .

# Install dependencies in a virtual environment
RUN python -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"
RUN pip install --no-cache-dir --upgrade pip &&
    pip install --no-cache-dir -r requirements.txt

# Stage 2: Runtime - Minimal production image
FROM python:3.11-slim

WORKDIR /app

# Copy virtual environment from builder
COPY --from=builder /opt/venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Security: Create non-root user
RUN useradd --create-home --shell /bin/bash appuser

# Copy application code
COPY artifacts/app/ /app/

# Ensure database directory is writable
RUN chown -R appuser:appuser /app

# Switch to non-root user
USER appuser

# Python optimization flags
ENV PYTHONUNBUFFERED=1
    PYTHONDONTWRITEBYTECODE=1

# Expose application port
EXPOSE 8000

# Run application
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"]
```

Generate the complete, production-ready Dockerfile now:"""

print("--- Generating Dockerfile ---")
dockerfile_content = get_completion(dockerfile_prompt, client, model_name, api_provider)
cleaned_dockerfile = clean_llm_output(dockerfile_content, language='dockerfile')
print(cleaned_dockerfile)

if cleaned_dockerfile:
    save_artifact(cleaned_dockerfile, "Dockerfile")

--- Generating Dockerfile ---
# For reference, a good .dockerignore file would contain:
# .git
# .gitignore
# .dockerignore
# __pycache__/
# *.pyc
# *.pyo
# *.pyd
# .pytest_cache/
# .venv/
# venv/
# env/

# ---- Stage 1: Builder ----
# This stage installs dependencies into a virtual environment.
FROM python:3.11-slim AS builder

# Set environment variables for Python
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

# Set the working directory
WORKDIR /opt/venv

# Create a virtual environment
RUN python -m venv .
ENV PATH="/opt/venv/bin:$PATH"

# Upgrade pip and install wheel to build packages efficiently
RUN pip install --no-cache-dir --upgrade pip wheel

# Copy requirements file and install dependencies
# This is done in a separate layer to leverage Docker's caching mechanism.
# The layer will only be rebuilt if requirements.txt changes.
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt


# ---- Stage 2: Runtime ----
# This stage creates the final, minima

### Challenge 3 (Advanced): Generating the GitHub Actions Workflow

**Task:** Generate a complete GitHub Actions workflow file to automate the build and test process.

**Instructions:**
1.  Write a prompt to generate a GitHub Actions workflow file named `ci.yml`.
2.  Specify the following requirements for the workflow:
    * It should trigger on any `push` to the `main` branch.
    * It should define a single job named `build-and-test` that runs on `ubuntu-latest`.
    * The job should have steps to: 1) Check out the code, 2) Set up a Python environment, 3) Install dependencies from `requirements.txt`, and 4) Run the test suite using `pytest`.
3.  Save the generated YAML file to `.github/workflows/ci.yml`.

In [5]:
# Detailed prompt to generate a GitHub Actions CI workflow file
ci_workflow_prompt = """You are a DevOps and CI/CD expert specializing in GitHub Actions. Generate a production-ready, optimized GitHub Actions workflow file for continuous integration of a FastAPI Python application.

**Project Context:**
- FastAPI application with SQLAlchemy database
- Application code located in `artifacts/app/` directory
- Test suite located in `artifacts/tests/` directory with pytest
- Dependencies managed in `requirements.txt`
- Tests use pytest with fixtures (conftest.py) and in-memory database
- Python version: 3.11

**Task Requirements:**

1. **Workflow Triggers:**
   - Trigger on `push` events to the `main` branch
   - Also trigger on `pull_request` events targeting `main` (best practice for PR validation)
   - Allow manual workflow dispatch for on-demand runs

2. **Job Configuration:**
   - Job name: `build-and-test`
   - Runner: `ubuntu-latest` (latest stable Ubuntu environment)
   - Use concurrency groups to cancel redundant runs on the same branch

3. **Required Steps (in order):**
   
   **Step 1: Checkout Code**
   - Use `actions/checkout@v4` (latest stable version)
   - Fetch full git history for better context
   
   **Step 2: Set Up Python Environment**
   - Use `actions/setup-python@v5` (latest stable version)
   - Python version: `3.11`
   - Include caching for pip dependencies to speed up builds
   
   **Step 3: Install Dependencies**
   - Upgrade pip to latest version first
   - Install dependencies from `requirements.txt`
   - Use `--no-cache-dir` flag for consistency with Docker builds
   
   **Step 4: Run Test Suite**
   - Execute pytest on the `artifacts/tests/` directory
   - Use verbose output (`-v`) for detailed test results
   - Generate coverage report (if applicable)
   - Ensure proper exit codes for CI failure detection

4. **GitHub Actions Best Practices:**
   - Use semantic, descriptive step names
   - Include comments explaining each major section
   - Use proper YAML formatting and indentation (2 spaces)
   - Set timeout for the job (e.g., 10 minutes) to prevent hanging builds
   - Use environment variables where appropriate
   - Include proper error handling (fail-fast behavior)

5. **Performance Optimizations:**
   - Cache pip dependencies using `actions/setup-python`'s built-in caching
   - This significantly speeds up subsequent workflow runs
   - Cache key should be based on `requirements.txt` content hash

6. **Test Execution Details:**
   - Tests are in `artifacts/tests/` directory
   - Tests include: `test_main_simple.py`, `test_main_with_fixture.py`, `test_edge_cases.py`
   - Tests use `conftest.py` for pytest fixtures
   - Pytest command should target the test directory: `pytest artifacts/tests/ -v`

**Expected Output Format:**
Generate ONLY the ci.yml file contents. No explanations before or after, no markdown code blocks wrapper. Include helpful inline comments in the YAML file itself. The output should be ready to save directly to `.github/workflows/ci.yml`.

**Example of desired format and structure:**
```yaml
name: CI Pipeline

# Trigger workflow on push or pull request to main branch
on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]
  workflow_dispatch:  # Allow manual triggers

# Cancel redundant runs for the same branch
concurrency:
  group: ${{ github.workflow }}-${{ github.ref }}
  cancel-in-progress: true

jobs:
  build-and-test:
    name: Build and Test Application
    runs-on: ubuntu-latest
    timeout-minutes: 10
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v4
      
      - name: Set up Python 3.11
        uses: actions/setup-python@v5
        with:
          python-version: '3.11'
          cache: 'pip'  # Cache pip dependencies for faster builds
      
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install --no-cache-dir -r requirements.txt
      
      - name: Run test suite
        run: |
          pytest artifacts/tests/ -v
```

Generate the complete, production-ready GitHub Actions workflow file now:"""

print("--- Generating GitHub Actions Workflow ---")
ci_workflow_content = get_completion(ci_workflow_prompt, client, model_name, api_provider)
cleaned_yaml = clean_llm_output(ci_workflow_content, language='yaml')
print(cleaned_yaml)

if cleaned_yaml:
    # Note: The save_artifact helper creates directories as needed.
    save_artifact(cleaned_yaml, ".github/workflows/ci.yml")

--- Generating GitHub Actions Workflow ---
name: Python FastAPI CI

# This workflow defines the Continuous Integration (CI) pipeline for the FastAPI application.
# It runs on pushes and pull requests to the main branch, and can also be triggered manually.

on:
  # Trigger on push events to the main branch
  push:
    branches: [ "main" ]
  # Trigger on pull request events targeting the main branch
  pull_request:
    branches: [ "main" ]
  # Allow manual triggering of the workflow from the GitHub Actions UI
  workflow_dispatch:

# Concurrency configuration ensures that for a given branch, only the latest workflow run
# will proceed, cancelling any older, in-progress runs. This saves runner minutes.
concurrency:
  group: ${{ github.workflow }}-${{ github.ref }}
  cancel-in-progress: true

jobs:
  # The main job for building, linting, and testing the application.
  build-and-test:
    name: Build and Test Application
    # Use the latest stable version of Ubuntu provided by GitHub.
    r

## Lab Conclusion

Excellent! You have now generated a complete, professional Continuous Integration pipeline using AI. You created the dependency list, the containerization configuration, and the automation workflow, all from simple prompts. This is a powerful demonstration of how AI can automate complex DevOps tasks, allowing teams to build and ship software with greater speed and confidence.

> **Key Takeaway:** AI is a powerful tool for generating 'Configuration as Code' artifacts. By prompting for specific formats like `requirements.txt`, `Dockerfile`, or `ci.yml`, you can automate the creation of the files that define your entire build, test, and deployment processes.